In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pandas
import matplotlib.pyplot as plt

In [ ]:
import helpers

In [ ]:
mnist_dataset = helpers.MnistDataset()

In [ ]:
mnist_dataset.plot_image(9)

In [ ]:
mnist_dataset[400]

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        # initialise parent pytorch class
        super().__init__()

        # define neural network layers
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(0.02),
            nn.LayerNorm(200),
            nn.Linear(200, 10),
            nn.Sigmoid(),
        )

        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, using simple stochastic gradient descent
        self.optimiser = torch.optim.Adam(self.parameters())

        # counter and accumulator for progress
        self.counter = 0
        self.progress = []

        pass

    def forward(self, inputs):
        # simply run model
        return self.model(inputs)

    def train(self, inputs, targets):
        # calculate the output of the network
        outputs = self.forward(inputs)

        # calculate loss
        loss = self.loss_function(outputs, targets)

        # increase counter and accumulate error every 10
        self.counter += 1
        if self.counter % 10 == 0:
            self.progress.append(loss.item())
            pass
        if self.counter % 10000 == 0:
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, and update the weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()

        pass

    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=["loss"])
        df.plot(
            ylim=(0, 1.0),
            figsize=(16, 8),
            alpha=0.1,
            marker=".",
            grid=True,
            yticks=(0, 0.25, 0.5),
        )
        pass

    pass

In [ ]:
%%time
# create neural network

classifier = Classifier()

# train network on MNIST data set

epochs = 3

for i in range(epochs):
    print("training epoch", i + 1, "of", epochs)
    for label, image_data_tensor, target_tensor in mnist_dataset:
        classifier.train(image_data_tensor, target_tensor)
        pass
    pass

In [ ]:
classifier.plot_progress()

In [ ]:
record = 19
mnist_dataset.plot_image(record)

In [ ]:
image_data = mnist_dataset[record][1]
output = classifier.forward(image_data)
pandas.DataFrame(output.detach().numpy()).plot(kind="bar", legend=False, ylim=(0, 1))

In [ ]:
score = 0
items = 0

for label, image_data_tensor, target_tensor in mnist_dataset:
    answer = classifier.forward(image_data_tensor).detach().numpy()
    if answer.argmax() == label:
        score += 1
        pass
    items += 1
    pass

print(score, items, score / items)